In [1]:
# Modules to support development
import os
import re
import collections
import itertools
import functools
import logging
import pprint
import numpy as np
import heapq
import copy
from tqdm import tqdm

In [16]:
def read_input(puzzle_input):
    with open(puzzle_input) as ff:
        dd = ff.readlines()

    initial = {}
    operators = {}
    for ll in dd:
        mm = re.match("(\w+): (\d+)", ll.strip())
        if mm:
            initial[mm.group(1)] = int(mm.group(2))

        mm = re.match("(\w+): (\w+) ([\+\-\*\/]) (\w+)", ll.strip())
        if mm:
            if mm.group(3) == "+":
                func = lambda x, y: x + y
            elif mm.group(3) == "-":
                func = lambda x, y: x - y
            elif mm.group(3) == "*":
                func = lambda x, y: x * y
            elif mm.group(3) == "/":
                func = lambda x, y: x // y
            operators[(mm.group(2), mm.group(4))] = (mm.group(1), func)

    return initial, operators

def test_read_input():
    initial, operators = read_input(os.path.join(os.path.join("..", "dat", "day21_test.txt")))
    print(initial, operators)

test_read_input()

{'dbpl': 5, 'zczc': 2, 'dvpt': 3, 'lfqf': 4, 'humn': 5, 'ljgn': 2, 'sllz': 4, 'hmdt': 32} {('pppw', 'sjmn'): ('root', <function read_input.<locals>.<lambda> at 0x000002C601953520>), ('sllz', 'lgvd'): ('cczh', <function read_input.<locals>.<lambda> at 0x000002C6026A4670>), ('humn', 'dvpt'): ('ptdq', <function read_input.<locals>.<lambda> at 0x000002C6026A5240>), ('drzm', 'dbpl'): ('sjmn', <function read_input.<locals>.<lambda> at 0x000002C6026A52D0>), ('cczh', 'lfqf'): ('pppw', <function read_input.<locals>.<lambda> at 0x000002C6026A5360>), ('ljgn', 'ptdq'): ('lgvd', <function read_input.<locals>.<lambda> at 0x000002C6026A53F0>), ('hmdt', 'zczc'): ('drzm', <function read_input.<locals>.<lambda> at 0x000002C6026A5480>)}


In [22]:
def part1(puzzle_input):
    initial, operators = read_input(puzzle_input)

    state = {}
    for nn, vv in initial.items():
        state[nn] = vv

    while len(operators) > 0:
        for (xx, yy), (oo, ff) in list(operators.items()):
            if xx in state and yy in state:
                state[oo] = ff(state[xx], state[yy])
                operators.pop((xx, yy))

            if "root" in state:
                return state["root"]

def test_part1():
    ans = part1(os.path.join(os.path.join("..", "dat", "day21_test.txt")))
    print(ans)
    assert ans == 152

test_part1()

ans = part1(os.path.join(os.path.join("..", "dat", "day21.txt")))
print(ans)

152
158661812617812


In [80]:
def part2(puzzle_input):
    initial, _operators = read_input(puzzle_input)

    ii = 1

    # Figure out the sensitivity
    history = []
    sensitivity = None
    while True:
        state = {}
        operators = copy.deepcopy(_operators)
        for nn, vv in initial.items():
            state[nn] = vv

        state["humn"] = ii
        while len(operators) > 0:
            for (xx, yy), (oo, ff) in list(operators.items()):
                if xx in state and yy in state:
                    if oo == "root":
                        state[oo] = state[xx] - state[yy], state[xx], state[yy]
                    else:
                        state[oo] = ff(state[xx], state[yy])
                    operators.pop((xx, yy))

                if "root" in state:
                    print("Result", ii, state['root'])
                    break

        if state["root"][0] == 0:
            return ii

        # 2 to 1 sensitivity
        history.append((ii, state["root"][0]))

        if len(history) < 2:
            ii += 1
        elif history[-1][1] != history[0][1] and sensitivity is None:
            x_delta = history[-1][0] -  history[0][0]
            y_delta = history[-1][1] -  history[0][1]
            sensitivity = y_delta / x_delta
            print(sensitivity, x_delta, y_delta, history[-1], history[-2])
            if sensitivity == 0:
                ii += 1
                sensitivity = None
            else:
                ii += int(-1*history[-1][1] / sensitivity)
        elif sensitivity is not None:
            delta = -1*history[-1][1] / sensitivity
            if 0 <= delta < 1:
                delta = 1
            elif 0 > delta >= -1:
                delta = 1
            delta = int(delta)
            ii += delta
        else:
            ii += 1
    return None

def test_part2():
    ans = part2(os.path.join(os.path.join("..", "dat", "day21_test.txt")))
    print(ans)
    #assert ans == 301

test_part2()

ns = part2(os.path.join(os.path.join("..", "dat", "day21.txt")))#print(ans)

Result 1 (-150, 0, 150)
Result 2 (-150, 0, 150)
Result 3 (-149, 1, 150)
0.5 2 1 (3, -149) (2, -150)
Result 301 (0, 150, 150)
301
Result 1 (53229630478972, 105945721566758, 52716091087786)
Result 2 (53229630478972, 105945721566758, 52716091087786)
Result 3 (53229630478934, 105945721566720, 52716091087786)
-19.0 2 -38 (3, 53229630478934) (2, 53229630478972)
Result 2801559498894 (8752731789778, 61468822877564, 52716091087786)
Result 3262229593092 (1439241886430, 54155332974216, 52716091087786)
Result 3337979166062 (236659508970, 52952750596756, 52716091087786)
Result 3350434929692 (38914739558, 52755005827344, 52716091087786)
Result 3352483073879 (6398884882, 52722489972668, 52716091087786)
Result 3352819857293 (1052190726, 52717143278512, 52716091087786)
Result 3352875235752 (173015392, 52716264103178, 52716091087786)
Result 3352884341825 (28449484, 52716119537270, 52716091087786)
Result 3352885839166 (4678082, 52716095765868, 52716091087786)
Result 3352886085380 (769250, 52716091857036,